In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import (
    EarlyStopping,
    TensorBoard,
    ModelCheckpoint
)

# Building Models with TF 2.0

## 20 November 2019

# The TensorFlow Ecosystem

![tensorflow-ecosystem](assets/tf_ecosystem.png)

Source: [Josh Gordon, Google](https://youtu.be/5ECD8J3dvDQ?t=128)

# Strategies for Building Models

- Sequential API
- Functional API
- Functional API + custom layers / metrics / losses
- Subclass `tf.keras.Model` to fit your own needs

# Sequential API

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='sigmoid'))

With the Sequential API (or functional, or with subclassing), we're defining a data structure — this gives us the benefit of compile-time checks, such as ensuring that all of our layers are compatible with one another

# Functional API

Where the Sequential API is a stack of layers, the functional API is a **graph** of layers.

Why does this matter? Consider a couple of use cases:
  - Given an image, classify the animal in the image (sounds like a CNN)
  - Given an image of two dogs, identify the color of the dog on the right (CNN for the image; embedding + LSTM for the text question)

# Functional API

![Wide and Deep recommender system](assets/wide_deep.png)
(Source: [arXiv:1606.07792](https://arxiv.org/abs/1606.07792))

# Functional API

In [ ]:
wide_input = tf.keras.layers.Input(shape=(100,))
deep_input = tf.keras.layers.Input(shape=(100,))

deep_hidden = tf.keras.layers.Dense(64, activation='relu')(deep_input)
deep_hidden = tf.keras.layers.Dense(32, activation='relu')(deep_hidden)

combined = tf.keras.layers.concatenate([wide_input, deep_hidden])
output = tf.keras.layers.Dense(1, activation='sigmoid')(combined)

# Functional API

In [ ]:
model = tf.keras.Model(
    inputs=[wide_input, deep_input],
    outputs=output)
model.summary()

```
Model: "sample_model"
______________________________________________________________________________
Layer (type)                  Output Shape       Param #   Connected to                     
==============================================================================
input_1 (InputLayer)          [(None, 100)]      0                                          
______________________________________________________________________________
input_2 (InputLayer)          [(None, 100)]      0                                            
______________________________________________________________________________
dense_1 (Dense)               (None, 64)         6464      input_2[0][0]                   
______________________________________________________________________________
dense_2 (Dense)               (None, 32)         2080      dense_1[0][0]                   
______________________________________________________________________________
concatenate_1 (Concatenate)   (None, 132)        0         input_1[0][0]                   
                                                           dense_2[0][0]                   
______________________________________________________________________________
dense_3 (Dense)               (None, 1)          133       concatenate_1[0][0]              
==============================================================================
Total params: 8,677
Trainable params: 8,677
Non-trainable params: 0
```

# Functional API

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='assets/model_dag.png')

![](assets/model_dag.png)

# Subclassing

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='sample_model')
        self.dense_1 = tf.keras.layers.Dense(32)
        self.dense_2 = tf.keras.layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = tf.nn.relu(x)
        return self.dense_2(x)

# Model Training
## From simple to arbitrarily complex

  - `model.fit()`
  - `model.fit()` with callbacks
  - `model.train_on_batch()` with callbacks
  - Roll your own training loop with `GradientTape()`

# Model Training

In [ ]:
model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[
        EarlyStopping(),
        TensorBoard(),
        ModelCheckpoint()
    ]
)

# Let's build a model

Access the Google Colab notebook here: [https://go.c18l.org/u/tf2](https://go.c18l.org/u/tf2)